# Transfer learning and fine-tuning

#

Model wstępnie wytrenowany to zapisana sieć, która została wcześniej przeszkolona na dużym zbiorze danych, zwykle w zadaniu klasyfikacji obrazów na dużą skalę. Możesz użyć gotowego modelu lub skorzystać z transferu uczenia się, aby dostosować ten model do określonego zadania.

Intuicja stojąca za transferem uczenia się w klasyfikacji obrazów polega na tym, że jeśli model jest trenowany na wystarczająco dużym i ogólnym zbiorze danych, to model ten będzie skutecznie służył jako ogólny model świata wizualnego. Możesz wtedy skorzystać z tych nauczonych map cech bez konieczności rozpoczynania od zera i trenowania dużego modelu na dużym zbiorze danych.

W tym notatniku spróbujesz dwóch sposobów dostosowania wstępnie wytrenowanego modelu:

Ekstrakcja cech: Wykorzystaj reprezentacje nauczone przez wcześniejszą sieć do ekstrakcji znaczących cech z nowych próbek. Po prostu dodaj nowy klasyfikator, który zostanie wytrenowany od podstaw na wierzchu wstępnie wytrenowanego modelu, dzięki czemu możesz przystosować mapy cech wcześniej nauczone do zbioru danych. Nie musisz (ponownie) trenować całego modelu. Bazowa sieć konwolucyjna już zawiera funkcje, które są ogólnie przydatne do klasyfikacji obrazów. Jednak ostatnia, kwalifikująca część wstępnie wytrenowanego modelu jest specyficzna dla pierwotnego zadania klasyfikacji i następnie specyficzna dla zestawu klas, na których model był trenowany.

Dopasowanie modelu: Odmroź kilka z najwyższych warstw zamrożonego modelu bazowego i jednocześnie trenuj zarówno nowo dodane warstwy klasyfikatora, jak i ostatnie warstwy modelu bazowego. Pozwala to nam na "dostrojenie" reprezentacji cech wyższego rzędu w modelu bazowym, aby były bardziej istotne dla konkretnego zadania. Będziesz postępować zgodnie z ogólnym procesem uczenia maszynowego.

1. Sprawdź i zrozum dane.
2. Utwórz potok wejściowy, w tym przypadku za pomocą Keras ImageDataGenerator.
3. Skomponuj model.
4. Załaduj wstępnie wytrenowany model bazowy (i wstępnie wytrenowane wagi).
5. Dołóż na wierzch warstwy klasyfikatora.
6. Trenuj model.
7. Oceń

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

## Data preprocessing



### Data download

Bedziemy korzystać z zestawu danych zawierającego kilka tysięcy obrazów kotów i psów. Pobierz i rozpakuj plik zip zawierający obrazy, a następnie utwórz zbiór danych tf.data.Dataset do szkolenia i walidacji za pomocą narzędzia tf.keras.utils.image_dataset_from_directory.

In [ ]:
path_to_zip

In [ ]:
# Pobieranie zbioru danych i tworzenie katalogów z danymi treningowymi i walidacyjnymi
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

In [ ]:
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

In [ ]:
# Definiowanie rozmiaru paczki (batch size) i rozmiaru obrazów
BATCH_SIZE = 32
IMG_SIZE = (160, 160)

# Tworzenie zbioru danych treningowych przy użyciu funkcji image_dataset_from_directory
train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)


In [ ]:
# Tworzenie zbioru danych walidacyjnych przy użyciu funkcji image_dataset_from_directory
validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE)


Wyświetl pierwszych dziewięć obrazów i etykiet z zestawu treningowego:





In [ ]:
# Pobieranie nazw klas z zbioru danych treningowych i wizualizacja losowych obrazów
class_names = train_dataset.class_names

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")


Ponieważ oryginalny zestaw danych nie zawiera zbioru testowego, będziesz musiał/a go utworzyć. Aby to zrobić, określ, ile partii danych jest dostępnych w zestawie walidacyjnym za pomocą tf.data.experimental.cardinality, a następnie przenieś 20% z nich do zbioru testowego.

In [ ]:
# Dzielenie zbioru danych walidacyjnych na zbiór walidacyjny i testowy
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)


In [ ]:
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

Użyj buforowanego wczytywania wstępnego, aby ładować obrazy z dysku bez blokowania operacji I/O.

In [ ]:
# Automatyczne dopasowanie wielkości bufora
AUTOTUNE = tf.data.AUTOTUNE

# Wstępne ładowanie danych treningowych, walidacyjnych i testowych
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)


### Use data augmentation

Gdy nie dysponujesz dużym zbiorem danych obrazowych, dobrą praktyką jest sztuczne wprowadzenie różnorodności próbek poprzez stosowanie losowych, ale realistycznych transformacji na obrazach treningowych, takich jak obrót i lustrzane odbicie poziome. Pomaga to modelowi poznać różne aspekty danych treningowych i zmniejszyć przypadkowe dopasowanie.

In [ ]:
# Definicja sekwencji przekształceń danych
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])


Uwaga: te warstwy są aktywne tylko podczas trenowania, gdy wywołujesz Model.fit. Są one nieaktywne, gdy model jest używany w trybie wnioskowania w Model.evaluate lub Model.predict.

Zastosujmy te warstwy wielokrotnie do tego samego obrazu i zobaczmy wynik.


In [ ]:
# Wizualizacja przekształceń danych treningowych
for image, _ in train_dataset.take(1):
  plt.figure(figsize=(10, 10))
  first_image = image[0]
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
    plt.imshow(augmented_image[0] / 255)
    plt.axis('off')


###Zmiana skali wartości pikseli

 Za chwilę pobierzesz tf.keras.applications.MobileNetV2 do użycia jako modelu bazowego. Ten model oczekuje wartości pikseli w przedziale [-1, 1], ale w tej chwili wartości pikseli na twoich obrazach są w przedziale [0, 255]. Aby zmienić ich skalę, użyj metody przetwarzania wstępnego zawartej w modelu.

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

### Utwórz model bazowy z wstępnie szkolonych konwolucyjnych sieci neuronowych
Stworzysz model bazowy z modelu MobileNet V2 opracowanego przez Google. Jest to model wstępnie szkolony na zbiorze danych ImageNet, dużym zbiorze danych składającym się z 1,4 miliona obrazów i 1000 klas. ImageNet to zbiór danych szkoleniowych z szeroką gamą kategorii, takich jak jackfruit i syringe. Ta podstawa wiedzy pomoże nam klasyfikować koty i psy z naszego konkretnego zbioru danych.

Najpierw musisz wybrać, którą warstwę MobileNet V2 wykorzystasz do ekstrakcji cech. Ostatnia warstwa klasyfikacji (na "górze", ponieważ większość diagramów modeli uczenia maszynowego układa się od dołu do góry) nie jest bardzo przydatna. Zamiast tego, będziesz stosować powszechną praktykę polegającą na poleganiu na ostatniej warstwie przed operacją spłaszczania. Ta warstwa nazywa się "warstwą wąskiego gardła" (ang. bottleneck layer). Cechy warstwy wąskiego gardła zachowują więcej ogólności w porównaniu do ostatniej/najwyższej warstwy.

Najpierw utwórz model MobileNet V2, który jest wczytany z wstępnie wyuczonymi wagami na zbiorze danych ImageNet. Określając argument include_top=False, wczytujesz sieć, która nie zawiera warstw klasyfikacji na górze, co jest idealne do ekstrakcji cech.

In [ ]:
# Tworzenie bazowego modelu na podstawie wstępnie wytrenowanego modelu MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


Ten ekstraktor cech konwertuje każdy obraz 160x160x3 na blok cech 5x5x1280. Zobaczmy, co robi z przykładową partią obrazów:





In [ ]:
image_batch.shape

In [ ]:
# Obliczanie cech z obrazów treningowych przy użyciu bazowego modelu
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)


## Feature extraction
W tym kroku zamrozisz bazę konwolucyjną stworzoną w poprzednim kroku i będziesz używać jej jako ekstraktora cech. Dodatkowo, dodasz klasyfikator na wierzch i będziesz trenować klasyfikator najwyższego poziomu.





### Freeze the convolutional base

Przed skompilowaniem i trenowaniem modelu ważne jest, aby zamrozić bazę konwolucyjną. Zamrożenie (poprzez ustawienie layer.trainable = False) zapobiega aktualizacji wag w danej warstwie podczas treningu. MobileNet V2 ma wiele warstw, więc ustawienie flagi trainable całego modelu na False spowoduje zamrożenie wszystkich warstw.

In [ ]:
base_model.trainable = False

In [ ]:
base_model.summary()

#### Ważna uwaga dotycząca warstw BatchNormalization

Wiele modeli zawiera warstwy tf.keras.layers.BatchNormalization. Ta warstwa jest szczególnym przypadkiem i należy zachować ostrożność w kontekście dopasowywania modelu, jak pokazano później w tym samouczku.

Gdy ustawisz layer.trainable = False, warstwa BatchNormalization działa w trybie wnioskowania i nie aktualizuje swoich statystyk średniej i wariancji.

Gdy odblokowujesz model zawierający warstwy BatchNormalization w celu wykonania dopasowywania, powinieneś zachować warstwy BatchNormalization w trybie wnioskowania, przekazując training=False podczas wywoływania modelu bazowego. W przeciwnym razie aktualizacje stosowane do wag niemożliwych do trenowania zniszczą to, czego model się nauczył.

In [ ]:
base_model.summary()

### Dodaj głowicę klasyfikacji




Aby wygenerować predykcje z bloku cech, należy uśrednić położenia przestrzenne 5x5 za pomocą warstwy tf.keras.layers.GlobalAveragePooling2D, aby przekonwertować cechy na pojedynczy wektor 1280-elementowy dla każdego obrazu.

In [ ]:
feature_batch.shape

In [ ]:
feature_batch_average.shape

In [ ]:
# Tworzenie warstwy uśredniającej i obliczanie uśrednionych cech
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)


Zastosuj warstwę tf.keras.layers.Dense, aby przekonwertować te cechy na pojedynczą predykcję dla każdego obrazu. Tutaj nie potrzebujesz funkcji aktywacji, ponieważ ta predykcja będzie traktowana jako logit, czyli surowa wartość predykcji. Dodatnie liczby przewidują klasę 1, ujemne liczby przewidują klasę 0.

In [ ]:
# Tworzenie warstwy predykcji i obliczanie przewidywanych wartości
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)


Zbuduj model, łącząc warstwy augmentacji danych, skalowania, base_model i ekstraktora cech za pomocą Keras Functional API. Jak już wspomniano, użyj training=False, ponieważ nasz model zawiera warstwę BatchNormalization.

In [ ]:
# Tworzenie modelu z sekwencją warstw
inputs = tf.keras.Input(shape=(160, 160, 3))

# Definiowanie sekwencji przekształceń danych wejściowych
x = data_augmentation(inputs)

# Przetwarzanie obrazów wejściowych za pomocą funkcji preprocess_input
x = preprocess_input(x)

# Obliczanie cech z obrazów wejściowych za pomocą modelu MobileNetV2
x = base_model(x, training=False)

# Obliczanie uśrednionych cech z wyjść modelu MobileNetV2
x = global_average_layer(x)

# Zmniejszenie przetrenowania modelu za pomocą warstwy Dropout
x = tf.keras.layers.Dropout(0.2)(x)

# Obliczanie przewidywanych wartości dla danych wejściowych
outputs = prediction_layer(x)

# Tworzenie modelu z określonymi powyżej warstwami
model = tf.keras.Model(inputs, outputs)


In [ ]:
outputs

### Skompiluj model
Skompiluj model przed jego trenowaniem. Ponieważ mamy dwie klasy, użyj straty tf.keras.losses.BinaryCrossentropy z parametrem from_logits=True, ponieważ model dostarcza liniową wartość wyjściową.

In [ ]:
# Kompilacja modelu z określonymi metrykami i hiperparametrami uczenia
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
model.summary()

W MobileNet jest zamrożonych 2,5 miliona parametrów, ale w warstwie Dense jest 1,2 tysiąca parametrów trainable. Są one podzielone między dwa obiekty tf.Variable: wagi i obciążenia.

In [ ]:
len(model.trainable_variables)

### Wytrenuj model

Po przetrenowaniu przez 10 epok powinieneś zobaczyć około 94% dokładności na zbiorze walidacyjnym.

In [ ]:
# Określenie liczby epok oraz początkowej wartości funkcji kosztu i metryki dokładności klasyfikacji
initial_epochs = 10
loss0, accuracy0 = model.evaluate(validation_dataset)


In [ ]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

In [ ]:
# Trenowanie modelu na danych treningowych przez określoną liczbę epok
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)


In [ ]:
history.history['accuracy']

## Krzywe uczenia
Spójrzmy na krzywe uczenia dotyczące dokładności/strat w trakcie treningu i walidacji, gdy używamy bazy modelu MobileNetV2 jako ustalonego ekstraktora cech.

In [ ]:
# Wyświetlenie wykresów dokładności klasyfikacji i funkcji kosztu na danych treningowych i walidacyjnych
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


Uwaga: Jeśli zastanawiasz się, dlaczego miary walidacji są wyraźnie lepsze niż miary treningowe, głównym czynnikiem jest to, że warstwy takie jak tf.keras.layers.BatchNormalization i tf.keras.layers.Dropout wpływają na dokładność podczas treningu. Są one wyłączone podczas obliczania straty walidacyjnej.

W mniejszym stopniu jest to także dlatego, że miary treningowe raportują średnią wartość dla epoki, podczas gdy miary walidacyjne są obliczane po epoce, więc miary walidacyjne widzą model, który został przetrenowany nieco dłużej.

##Dopasowywanie modelu
W eksperymencie z ekstrakcją cech trenowałeś tylko kilka warstw na bazie modelu MobileNetV2. Wagi wstępnie wyuczonych sieci nie były aktualizowane podczas treningu.

Jednym ze sposobów dalszego zwiększenia wydajności jest trenowanie (lub "dopasowywanie") wag najlepszych warstw z wstępnie szkolenego modelu wraz z trenowaniem klasyfikatora, który dodałeś. Proces trenowania wymusi dopasowanie wag z map cech ogólnych do cech specyficznych dla zbioru danych.

Uwaga: Powinieneś spróbować dopasować wag tylko kilku najlepszych warstw, a nie całego modelu MobileNet. W większości sieci konwolucyjnych im wyższa warstwa, tym bardziej jest specjalizowana. Pierwsze kilka warstw uczy się bardzo prostych i ogólnych cech, które generalizują się do prawie wszystkich rodzajów obrazów. Im wyższa warstwa, tym cechy są coraz bardziej specyficzne dla zbioru danych, na którym model był szkolony. Celem dopasowania jest dostosowanie tych specjalizowanych cech do pracy z nowym zbiorem danych, a nie nadpisanie nauki ogólnych cech.

### Odblokuj górne warstwy modelu





Wystarczy odblokować base_model i ustawic dolne warstwy na niewytrenowane. Następnie trzeba przekompilować model (aby te zmiany zaczęły działać) i kontynuować trening.





In [ ]:
base_model.trainable = True

In [ ]:
# Wypisanie liczby warstw modelu bazowego
print("Number of layers in the base model: ", len(base_model.layers))


In [ ]:

# Fine-tuning od tej warstwy w górę
fine_tune_at = 100

# Zamrożenie wszystkich warstw przed warstwą "fine_tune_at"
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False


### Skompiluj model

Ponieważ trenujemy znacznie większy model i chcemy dostosować wstępnie szkoleniowe wagi, ważne jest, aby w tym etapie użyć niższej szybkości uczenia. W przeciwnym razie model może bardzo szybko zaczynać overfitting.

In [ ]:
# Ustawienie funkcji straty, optymalizatora i metryk dla modelu
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10),
              metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
history.epoch[-1]

### Kontynuuj trenowanie modelu




In [ ]:
# Fine-tuning modelu i kontynuacja treningu
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_dataset)


In [ ]:
acc

In [ ]:
# Obliczanie dokładności i straty podczas fine-tuningu
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']


In [ ]:
# Wizualizacja wyników fine-tuningu
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1], plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1], plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


### Ocena i predykcja




In [ ]:
# Ewaluacja modelu na zbiorze testowym
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)


In [ ]:

# Predykcja na zbiorze testowym
image_batch, label_batch = test_dataset.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch).flatten()

In [ ]:
predictions

In [ ]:


# Zastosowanie funkcji sigmoidalnej, ponieważ model zwraca logity
predictions = tf.nn.sigmoid(predictions)

In [ ]:
predictions

In [ ]:
predictions = tf.where(predictions < 0.5, 0, 1)

In [ ]:
predictions

In [ ]:


print('Predictions:\n', predictions.numpy())
print('Labels:\n', label_batch)

plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i].astype("uint8"))
  plt.title(class_names[predictions[i]])
  plt.axis("off")
